## Writing Classic CNN LeNet5 from Scratch in PyTorch

In this notebook, we would write one of the earliest Convolutional Neural Networks, LeNet5, from scratch in PyTorch. You can read more about it here: http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf

### Importing the libraries

Let's start by importing the required libraries and defining some required variables

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Downloading and Loading the Dataset

We will download the datasets from `torchvision` and load them into PyTorch. We will also aply some transformations, such as resizing the images, converting them to tensors and normalizing them using the mean and standard deviation

In [ ]:
!python test.py

0it [00:00, ?it/s]

  0%|                                               | 0/9912422 [00:00<?, ?it/s]

 14%|███▊                       | 1417216/9912422 [00:00<00:00, 13372405.29it/s]

 47%|████████████▋              | 4677632/9912422 [00:00<00:00, 16237172.57it/s]

 69%|██████████████████▋        | 6873088/9912422 [00:00<00:00, 17252987.25it/s]

...




0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...


/opt/conda/lib/python3.8/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:137.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


32768it [00:01, 31209.49it/s]
1654784it [00:00, 1690035.65it/s]                                               
8192it [00:00, 13435.85it/s]
9920512it [00:02, 4430700.02it/s]                                               


In [ ]:

#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root = './data',
                                              train = True,
                                                  transform = transforms.Compose([
                                                          transforms.Resize((32,32)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                                  download = False)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                                  train = False,
                                                  transform = transforms.Compose([
                                                          transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
               
                                           shuffle = True)

### LeNet5 From Scratch


In [ ]:
class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

### Setting Hyperparameters

In [ ]:

model = ConvNeuralNet( num_classes).to(device)

#Defining cost and optimizer
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

### Training

In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
            #Forward pass
        outputs = model(images)
        loss = cost(outputs, labels)
        	
        	# Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        		
        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))



Epoch [1/10], Step [400/938], Loss: 0.0604


Epoch [1/10], Step [800/938], Loss: 0.0857


Epoch [2/10], Step [400/938], Loss: 0.0082


Epoch [2/10], Step [800/938], Loss: 0.0119


Epoch [3/10], Step [400/938], Loss: 0.0110


Epoch [3/10], Step [800/938], Loss: 0.0154


Epoch [4/10], Step [400/938], Loss: 0.0149


Epoch [4/10], Step [800/938], Loss: 0.0438


### Testing

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
  
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
	 

Accuracy of the network on the 10000 test images: 98.81 %
